## Bài tập chương 3

21_LeTheKoi

Câu 1

In [63]:
import sqlite3
import math

In [64]:
# Kết nối đến cơ sở dữ liệu trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng để lưu dữ liệu cho A và B
cursor.execute('''
    CREATE TABLE cau_1 (
        id INTEGER PRIMARY KEY,
        A FLOAT,
        B FLOAT
    )
''')

# Thêm dữ liệu mẫu vào bảng
du_lieu = [
    (1, 2),
    (2, 4),
    (3, 6),
    (4, 8),
    (5, 10)
]

cursor.executemany('INSERT INTO cau_1 (A, B) VALUES (?, ?)', du_lieu)

# Lưu thay đổi
conn.commit()

In [65]:
# Viết câu lệnh SQL để tính hệ số tương quan Pearson
cau_lenh = '''
    SELECT 
        (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) / 
        SQRT((COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) * 
             (COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS he_so_tuong_quan
    FROM cau_1
'''

# Thực thi câu lệnh
cursor.execute(cau_lenh)
ket_qua = cursor.fetchone()[0]

# In kết quả
print(f"Hệ số tương quan (r_AB) là: {ket_qua}")
# Đóng kết nối
conn.close()

Hệ số tương quan (r_AB) là: 1.0


Câu 2

In [66]:
# Kết nối đến cơ sở dữ liệu trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng 
cursor.execute('''
    CREATE TABLE cau_2 (
        ngay TEXT,
        mau_xe TEXT,
        diem_so FLOAT
    )
''')
# Dữ liệu dạng quan hệ
du_lieu = [
    ('Day 1', 'A', 8),
    ('Day 1', 'B', 9),
    ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5),
    ('Day 2', 'B', 8.5),
    ('Day 2', 'C', 7),
    ('Day 3', 'A', 6),
    ('Day 3', 'B', 7),
    ('Day 3', 'C', 8),
    ('Day 4', 'A', 7),
    ('Day 4', 'B', 6),
    ('Day 4', 'C', 5)
]

# Thêm dữ liệu vào bảng
cursor.executemany('INSERT INTO cau_2 (ngay, mau_xe, diem_so) VALUES (?, ?, ?)', du_lieu)

# Lưu thay đổi
conn.commit()

In [67]:
# Tính trung bình tổng thể và trung bình của từng mẫu xe
cursor.execute('SELECT AVG(diem_so) FROM cau_2')
trung_binh_tong_the = cursor.fetchone()[0]

cursor.execute('''
    SELECT mau_xe, AVG(diem_so) AS trung_binh
    FROM cau_2
    GROUP BY mau_xe
''')
trung_binh_mau_xe = {row[0]: row[1] for row in cursor.fetchall()}

# Số quan sát và số nhóm
so_quan_sat = 12  # 4 ngày x 3 mẫu xe
so_nhom = 3  # 3 mẫu xe (A, B, C)
so_quan_sat_moi_nhom = 4  # 4 ngày mỗi mẫu xe

# Tính tổng bình phương giữa các nhóm (SSB)
ssb = 0
for mau_xe, tb in trung_binh_mau_xe.items():
    ssb += so_quan_sat_moi_nhom * (tb - trung_binh_tong_the) ** 2

# Tính tổng bình phương trong nhóm (SSW)
ssw = 0
cursor.execute('SELECT mau_xe, diem_so FROM cau_2')
for row in cursor.fetchall():
    mau_xe, diem = row
    ssw += (diem - trung_binh_mau_xe[mau_xe]) ** 2

# Tính giá trị F
msb = ssb / (so_nhom - 1)  # Phương sai giữa các nhóm
msw = ssw / (so_quan_sat - so_nhom)  # Phương sai trong nhóm
gia_tri_F = msb / msw if msw != 0 else float('inf')

# In kết quả
print("Kiểm tra sự khác biệt giữa các mẫu xe (ANOVA thủ công):")
print(f"Trung bình tổng thể: {trung_binh_tong_the:.2f}")
print("Trung bình của từng mẫu xe:")
for mau_xe, tb in trung_binh_mau_xe.items():
    print(f"Mẫu {mau_xe}: {tb:.2f}")
print(f"Tổng bình phương giữa các nhóm (SSB): {ssb:.2f}")
print(f"Tổng bình phương trong nhóm (SSW): {ssw:.2f}")
print(f"Giá trị F: {gia_tri_F:.2f}")

# So sánh với giá trị F-critical (tới hạn) cho df1 = 2, df2 = 9, alpha = 0.05
F_critical = 4.26  # Giá trị F-critical tra bảng cho df1 = 2, df2 = 9, alpha = 0.05
if gia_tri_F > F_critical:
    print("Có sự khác biệt đáng kể giữa các mẫu xe (F > F-critical).")
else:
    print("Không có sự khác biệt đáng kể giữa các mẫu xe (F <= F-critical).")

Kiểm tra sự khác biệt giữa các mẫu xe (ANOVA thủ công):
Trung bình tổng thể: 7.17
Trung bình của từng mẫu xe:
Mẫu A: 7.12
Mẫu B: 7.62
Mẫu C: 6.75
Tổng bình phương giữa các nhóm (SSB): 1.54
Tổng bình phương trong nhóm (SSW): 12.62
Giá trị F: 0.55
Không có sự khác biệt đáng kể giữa các mẫu xe (F <= F-critical).


In [68]:
# Phân loại điểm số thành "Thấp" (< 7) và "Cao" (≥ 7)
cursor.execute('''
    SELECT ngay, mau_xe, 
           CASE 
               WHEN diem_so < 7 THEN 'Thap'
               ELSE 'Cao'
           END AS phan_loai
    FROM cau_2
''')

# Tạo bảng tần số
bang_tan_so = {}
for ngay in ['Day 1', 'Day 2', 'Day 3', 'Day 4']:
    bang_tan_so[ngay] = {'A': {'Thap': 0, 'Cao': 0}, 'B': {'Thap': 0, 'Cao': 0}, 'C': {'Thap': 0, 'Cao': 0}}

for row in cursor.fetchall():
    ngay, mau_xe, phan_loai = row
    bang_tan_so[ngay][mau_xe][phan_loai] += 1

# Tính tổng theo ngày và mẫu xe
tong_theo_ngay = {'Day 1': 0, 'Day 2': 0, 'Day 3': 0, 'Day 4': 0}
tong_theo_mau_xe = {'A': {'Thap': 0, 'Cao': 0}, 'B': {'Thap': 0, 'Cao': 0}, 'C': {'Thap': 0, 'Cao': 0}}
tong_tat_ca = {'Thap': 0, 'Cao': 0}

for ngay in bang_tan_so:
    for mau_xe in bang_tan_so[ngay]:
        for phan_loai in bang_tan_so[ngay][mau_xe]:
            so_lan = bang_tan_so[ngay][mau_xe][phan_loai]
            tong_theo_ngay[ngay] += so_lan
            tong_theo_mau_xe[mau_xe][phan_loai] += so_lan
            tong_tat_ca[phan_loai] += so_lan

# Tính giá trị kỳ vọng và chi-squared
chi_squared = 0
for ngay in bang_tan_so:
    for mau_xe in bang_tan_so[ngay]:
        for phan_loai in bang_tan_so[ngay][mau_xe]:
            O = bang_tan_so[ngay][mau_xe][phan_loai]  # Giá trị quan sát
            E = (tong_theo_ngay[ngay] * tong_theo_mau_xe[mau_xe][phan_loai]) / (tong_tat_ca['Thap'] + tong_tat_ca['Cao'])  # Giá trị kỳ vọng
            if E > 0:
                chi_squared += (O - E) ** 2 / E

# In bảng tần số
print("\nBảng tần số (Thấp/Cao):")
for ngay in bang_tan_so:
    print(f"{ngay}:")
    for mau_xe in bang_tan_so[ngay]:
        print(f"  Mẫu {mau_xe}: Thấp = {bang_tan_so[ngay][mau_xe]['Thap']}, Cao = {bang_tan_so[ngay][mau_xe]['Cao']}")

# In kết quả kiểm định chi-squared
print(f"\nGiá trị chi-squared: {chi_squared:.2f}")
# Bậc tự do: (số ngày - 1) * (số mẫu xe - 1) = (4-1)*(3-1) = 6
# Giá trị tới hạn chi-squared cho df = 6, alpha = 0.05 là 12.59
chi_squared_critical = 12.59
if chi_squared > chi_squared_critical:
    print("Kết quả thử nghiệm phụ thuộc vào cả ngày và mẫu xe (chi-squared > chi-squared critical).")
else:
    print("Kết quả thử nghiệm không phụ thuộc vào ngày hoặc mẫu xe (chi-squared <= chi-squared critical).")

# Đóng kết nối
conn.close()


Bảng tần số (Thấp/Cao):
Day 1:
  Mẫu A: Thấp = 0, Cao = 1
  Mẫu B: Thấp = 0, Cao = 1
  Mẫu C: Thấp = 0, Cao = 1
Day 2:
  Mẫu A: Thấp = 0, Cao = 1
  Mẫu B: Thấp = 0, Cao = 1
  Mẫu C: Thấp = 0, Cao = 1
Day 3:
  Mẫu A: Thấp = 1, Cao = 0
  Mẫu B: Thấp = 0, Cao = 1
  Mẫu C: Thấp = 0, Cao = 1
Day 4:
  Mẫu A: Thấp = 0, Cao = 1
  Mẫu B: Thấp = 1, Cao = 0
  Mẫu C: Thấp = 1, Cao = 0

Giá trị chi-squared: 12.00
Kết quả thử nghiệm không phụ thuộc vào ngày hoặc mẫu xe (chi-squared <= chi-squared critical).


Câu 3

In [69]:
# Kết nối đến cơ sở dữ liệu trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng
cursor.execute('''
    CREATE TABLE cau_3 (
        id INTEGER PRIMARY KEY,
        departure_time INTEGER
    )
''')
# Dữ liệu mẫu
du_lieu = [
    (1, 830),   # 8:30 AM
    (2, 1445),  # 2:45 PM
    (3, 1130),  # 11:30 AM
    (4, 1700),  # 5:00 PM
    (5, 945),   # 9:45 AM
]

# Thêm dữ liệu vào bảng
cursor.executemany('INSERT INTO cau_3 (id, departure_time) VALUES (?, ?)', du_lieu)

# Lưu thay đổi
conn.commit()

In [70]:
# Tạo câu lệnh SQL để chuyển đổi thời gian
cursor.execute('''
    SELECT 
        id,
        departure_time,
        CASE
            WHEN (departure_time / 100) = 0 THEN '12'  -- Trường hợp 12:xx AM
            WHEN (departure_time / 100) > 12 THEN (departure_time / 100) - 12
            WHEN (departure_time / 100) = 12 THEN 12  -- Trường hợp 12:xx PM
            ELSE (departure_time / 100)
        END || ':' || 
        printf('%02d', departure_time % 100) || ' ' || 
        CASE
            WHEN (departure_time / 100) >= 12 THEN 'PM'
            ELSE 'AM'
        END AS thoi_gian_chuan
    FROM cau_3
''')

# Lấy và in kết quả
print("Kết quả chuyển đổi thời gian:")
for row in cursor.fetchall():
    id, departure_time, thoi_gian_chuan = row
    print(f"ID: {id}, Thời gian gốc: {departure_time}, Thời gian chuẩn: {thoi_gian_chuan}")

# Đóng kết nối
conn.close()

Kết quả chuyển đổi thời gian:
ID: 1, Thời gian gốc: 830, Thời gian chuẩn: 8:30 AM
ID: 2, Thời gian gốc: 1445, Thời gian chuẩn: 2:45 PM
ID: 3, Thời gian gốc: 1130, Thời gian chuẩn: 11:30 AM
ID: 4, Thời gian gốc: 1700, Thời gian chuẩn: 5:00 PM
ID: 5, Thời gian gốc: 945, Thời gian chuẩn: 9:45 AM


Câu 4

In [71]:
# Kết nối đến cơ sở dữ liệu trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng 
cursor.execute('''
    CREATE TABLE cau_4 (
        id INTEGER PRIMARY KEY,
        gia_tri FLOAT
    )
''')
# Dữ liệu 
du_lieu = [
    (1, 10),
    (2, 12),
    (3, 11),
    (4, 13),
    (5, 9),
    (6, 50),  
    (7, 11),
    (8, 14),
    (9, 8),
    (10, 100),  
]

# Thêm dữ liệu vào bảng
cursor.executemany('INSERT INTO cau_4 (id, gia_tri) VALUES (?, ?)', du_lieu)

# Lưu thay đổi
conn.commit()

In [72]:
# Giả định k = 3 
k = 3

# Sắp xếp dữ liệu và lấy giá trị ở giữa
cursor.execute('SELECT COUNT(*) FROM cau_4')
so_luong = cursor.fetchone()[0]

if so_luong % 2 == 0:
    # Nếu số lượng chẵn, lấy trung bình của 2 giá trị ở giữa
    cursor.execute('''
        SELECT AVG(gia_tri) 
        FROM (
            SELECT gia_tri 
            FROM cau_4 
            ORDER BY gia_tri 
            LIMIT 2 OFFSET (SELECT (COUNT(*) - 2) / 2 FROM cau_4)
        )
    ''')
else:
    # Nếu số lượng lẻ, lấy giá trị ở giữa
    cursor.execute('''
        SELECT gia_tri 
        FROM cau_4 
        ORDER BY gia_tri 
        LIMIT 1 OFFSET (SELECT (COUNT(*) - 1) / 2 FROM cau_4)
    ''')

trung_vi = cursor.fetchone()[0]
print(f"Trung vị của dữ liệu: {trung_vi}")


# Tạo bảng tạm để lưu độ lệch tuyệt đối
cursor.execute('''
    CREATE TEMP TABLE do_lech AS
    SELECT id, gia_tri, ABS(gia_tri - ?) AS do_lech_tuyet_doi
    FROM cau_4
''', (trung_vi,))

# Tính trung vị của độ lệch tuyệt đối (MAD)
cursor.execute('SELECT COUNT(*) FROM do_lech')
so_luong_do_lech = cursor.fetchone()[0]

if so_luong_do_lech % 2 == 0:
    cursor.execute('''
        SELECT AVG(do_lech_tuyet_doi)
        FROM (
            SELECT do_lech_tuyet_doi
            FROM do_lech
            ORDER BY do_lech_tuyet_doi
            LIMIT 2 OFFSET (SELECT (COUNT(*) - 2) / 2 FROM do_lech)
        )
    ''')
else:
    cursor.execute('''
        SELECT do_lech_tuyet_doi
        FROM do_lech
        ORDER BY do_lech_tuyet_doi
        LIMIT 1 OFFSET (SELECT (COUNT(*) - 1) / 2 FROM do_lech)
    ''')

mad = cursor.fetchone()[0]
print(f"MAD (Median Absolute Deviation): {mad}")


nguong = k * mad
print(f"Ngưỡng để xác định ngoại lệ (|x - trung vị| > {k} * MAD): {nguong}")

cursor.execute('''
    SELECT id, gia_tri
    FROM do_lech
    WHERE do_lech_tuyet_doi > ?
''', (nguong,))

# In kết quả
print("\nCác giá trị ngoại lệ:")
for row in cursor.fetchall():
    id, gia_tri = row
    print(f"ID: {id}, Giá trị: {gia_tri}")

# Đóng kết nối
conn.close()

Trung vị của dữ liệu: 11.5
MAD (Median Absolute Deviation): 2.0
Ngưỡng để xác định ngoại lệ (|x - trung vị| > 3 * MAD): 6.0

Các giá trị ngoại lệ:
ID: 6, Giá trị: 50.0
ID: 10, Giá trị: 100.0


Câu 5

In [73]:
# Kết nối đến cơ sở dữ liệu trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng 
cursor.execute('''
    CREATE TABLE cau_5 (
        id INTEGER PRIMARY KEY,
        last_name TEXT,
        weight FLOAT,
        height FLOAT
    )
''')
# Dữ liệu 
du_lieu = [
    (1, 'Nguyen', 70.5, 170),
    (2, 'Nguyen', 71.0, 165),  
    (3, 'Tran', 65.0, 160),
    (4, 'Nguyen', 80.0, 175), 
    (5, 'Tran', 88.5, 168),
    (6,'Le',50,165),
    (7,'Pham',60,169),
    (8,'Pham',70,170),
    (9,'Tran',89,165)
]

# Thêm dữ liệu vào bảng
cursor.executemany('INSERT INTO cau_5 (id, last_name, weight, height) VALUES (?, ?, ?, ?)', du_lieu)

# Lưu thay đổi
conn.commit()

In [74]:
# Định nghĩa ngưỡng cho weight (chênh lệch tối đa 5 kg)
nguong_weight = 5

# Truy vấn SQL để tìm các cặp có thể là cùng một người
cursor.execute('''
    SELECT 
        p1.id AS id_1, 
        p1.last_name AS ho_1, 
        p1.weight AS can_nang_1,
        p2.id AS id_2, 
        p2.last_name AS ho_2, 
        p2.weight AS can_nang_2
    FROM cau_5 p1
    JOIN cau_5 p2
    ON p1.id < p2.id  -- Tránh so sánh trùng lặp (p1 với p2 và p2 với p1)
    WHERE 
        p1.last_name = p2.last_name  -- Điều kiện 1: last_name giống nhau
        AND ABS(p1.weight - p2.weight) <= ?  -- Điều kiện 2: chênh lệch weight <= 5
''', (nguong_weight,))

# In kết quả
print("Các cặp có thể là cùng một người (dựa trên khoảng cách kết hợp Boolean):")
for row in cursor.fetchall():
    id_1, ho_1, can_nang_1, id_2, ho_2, can_nang_2 = row
    print(f"ID {id_1} (Họ: {ho_1}, Cân nặng: {can_nang_1}) và ID {id_2} (Họ: {ho_2}, Cân nặng: {can_nang_2})")

Các cặp có thể là cùng một người (dựa trên khoảng cách kết hợp Boolean):
ID 1 (Họ: Nguyen, Cân nặng: 70.5) và ID 2 (Họ: Nguyen, Cân nặng: 71.0)
ID 5 (Họ: Tran, Cân nặng: 88.5) và ID 9 (Họ: Tran, Cân nặng: 89.0)
